# Serve Multiple Fine-Tuned LoRA Adapters on a single endpoint using SageMaker LMI DLC

This notebook illustrates the deployment and service of multiple fine-tuned LoRA adapters employing a solitary base model copy on SageMaker, utilizing the DJL Serving Large Model Inference DLC. LoRA, or Low Rank Adapters, stands out as an efficacious method for refining large language models, considerably diminishing the count of trainable parameters in contrast to conventional fine-tuning, yet delivering analogous or superior results. For further insights into the LoRA technique, refer to this [paper](https://arxiv.org/abs/2106.09685).

One of the principal advantages of LoRA lies in the facility to seamlessly integrate and detach the fine-tuned adapters from the base model, enabling cost-effective and feasible adapter interchange at runtime. This notebook exemplifies the deployment of a SageMaker endpoint with a unified base model and multiple LoRA adapters and demonstrates how to modify adapters to cater to varying requests.

Given that LoRA adapters have a significantly smaller footprint compared to a base model (realistically being 100x-1000x smaller), it’s plausible to deploy an endpoint with one base model and several LoRA adapters utilizing substantially fewer hardware resources than would be the case with an equivalent number of comprehensively fine-tuned models.

The scenario elucidated in this notebook is inspired by the multi-adapter example found in HuggingFace's PEFT library: https://github.com/huggingface/peft/blob/main/examples/multi_adapter_examples/PEFT_Multi_LoRA_Inference.ipynb

### Install Packages and Import Dependencies

First, lets install required libraries such as `sagemaker`, `boto3`, `awscli` and `huggingface_hub`. You can safely ingore any warnming related to running pip as `root` user

In [1]:
!pip install huggingface_hub sagemaker boto3 awscli --upgrade --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Imports and set up

In [3]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from sagemaker.utils import name_from_base
from huggingface_hub import snapshot_download, notebook_login

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [39]:
sagemaker_session = sagemaker.session.Session()
s3_bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = (
    "hf-large-model-djl/lora-llama7b"  # folder within bucket where code artifact will go
)

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Download Model Artifacts and Upload to S3

We will be deploying an endpoint with 3 LoRA adapters. These are the models we will be using
- Base Model: https://huggingface.co/NousResearch/Llama-2-7b-hf
- LoRA Fine Tuned Adapter 1
- LoRA Fine Tuned Adapter 2
- LoRA Fine Tuned Adapter 3

The `base_model` directory contains all the base model artifacts from the corresponding repository on the huggingface hub. These model artifacts are generated from the `model.save_pretrained()` method of huggingface's transformers library.

Each of the adapters in the `adapters` directory contains the LoRA adapter artifacts. Typically there are two files: `adapter_model.bin` and `adapter_config.json` which are the adapter weights and adapter configuration respectively. These are typically obtained from the Peft library via the `PeftModel.save_pretrained()` method. In this example, the inference handler will register the adapters with names equivalent to their directory name, and we will use that name in the request to target a specific adapter.

```
|- base_model/
|--- <base-model-artifacts>/
|- adapters/
|--- <lora1>/
|-------- <lora1-model-artifacts>/
|--- <lora2>/
|-------- <lora2-model-artifacts>/
|--- <lora3>/
|-------- <lora3-model-artifacts>/
|--- <lora_n>/
|-------- <lora_n-model-artifacts>/
```

For the purposes of the this demonstration, we have packaged the model artifacts and uploaded it to s3. You will make a copy of the model artifact and deploy it to SageMaker endpoint

In [5]:
!aws s3 sync s3://{s3_bucket}/lora-llama7b-adapter s3://{s3_bucket}/lora-llama7b

## Creating Inference Handler and DJL Serving Configuration

The following files cover the model server configuration (`serving.properties`) and custom inference handler (`model.py`). This configuration can be used as an example to write your own inference handler for different models. 

In [7]:
!rm -rf lora-llama7b-code
!mkdir -p lora-llama7b-code

In [8]:
%%writefile lora-llama7b-code/serving.properties
# Python engine is currently the only engine supported for multi adapter use-case
engine=Python
option.model_id=s3://{{s3_bucket}}/lora-llama7b
option.base_model_path=base_model
option.adapters_path=adapters
option.dtype=fp16
option.entryPoint=model.py

Writing lora-llama7b-code/serving.properties


In [9]:
import jinja2

jinja_env = jinja2.Environment()
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("lora-llama7b-code/serving.properties").open().read())
Path("lora-llama7b-code/serving.properties").open("w").write(
    template.render(s3_bucket=s3_bucket)
)
!pygmentize lora-llama7b-code/serving.properties | cat -n

     1	# Python engine is currently the only engine supported for multi adapter use-case
     2	engine=Python
     3	option.model_id=s3://sagemaker-us-east-1-917092859813/lora-llama7b
     4	option.base_model_path=base_model
     5	option.adapters_path=adapters
     6	option.dtype=fp16
     7	option.entryPoint=model.py


This section walks through how to serve Python based model with DJL Serving.

1. Define Model
To get started, implement a python source file named model.py as the entry point. DJL Serving will run your request by invoking a handle function that you provide. The handle function should have the following signature:

    ```def handle(inputs: Input)```
    
2. If there are other packages you want to use with your script, you can include a requirements.txt file in the same directory with your model file to install other dependencies at runtime. A requirements.txt file is a text file that contains a list of items that are installed by using pip install. You can also specify the version of an item to install.

In [10]:
%%writefile lora-llama7b-code/model.py
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from peft import PeftModel
import torch
import os
from djl_python.inputs import Input
from djl_python.outputs import Output
import logging

model = None
tokenizer = None
logger = logging.getLogger()


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. 
        Write a response that appropriately completes the request. ### Instruction: {instruction} ### Input: {input} 
        ### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the 
        request.### Instruction: {instruction} ### Response:"""


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=256,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(torch.cuda.current_device())
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()


def load_model(base_model_path, adapters_path=None):
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    logging.info(f"Loaded Base Model {base_model_path}")

    print(os.listdir(adapters_path))
    if os.path.exists(adapters_path):
        first_adapter = True
        for adapter_dir in os.listdir(adapters_path):
            logging.info(f"Registering Adapter {os.path.join(adapters_path, adapter_dir)}")
            if not os.path.isdir(os.path.join(adapters_path, adapter_dir)):
                continue
            if first_adapter:
                model = PeftModel.from_pretrained(
                    model, os.path.join(adapters_path, adapter_dir), adapter_name=adapter_dir
                )
                first_adapter = False
            else:
                model.load_adapter(
                    os.path.join(adapters_path, adapter_dir), adapter_name=adapter_dir
                )
    return model, tokenizer


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        properties = inputs.get_properties()
        model_dir = properties.get("model_id")
        base_model_path = os.path.join(model_dir, properties.get("base_model_path"))
        adapters_path = os.path.join(model_dir, properties.get("adapters_path"))
        logging.info(base_model_path)
        logging.info(adapters_path)
        model, tokenizer = load_model(base_model_path, adapters_path=adapters_path)

    if inputs.is_empty():
        # initialization request
        return None

    json_inputs = inputs.get_as_json()
    sentence = json_inputs.get("inputs")
    adapter_name = json_inputs.get("adapter_name", None)
    generation_kwargs = json_inputs.get("parameters", {})
    if adapter_name is not None:
        model.set_adapter(adapter_name)
        outputs = evaluate(sentence, **generation_kwargs)
    else:
        with model.disable_adapter():
            outputs = evaluate(sentence, **generation_kwargs)

    return Output().add_as_json(outputs)

Writing lora-llama7b-code/model.py


DJL Serving supports model artifacts in model directory, .zip or .tar.gz format. To package model artifacts in a .tar.gz:

In [11]:
!rm -f model.tar.gz
!rm -rf lora-llama7b-code/.ipynb_checkpoints
!tar czvf model.tar.gz -C lora-llama7b-code .

./
./model.py
./serving.properties


## Create SageMaker Model and Endpoint

SageMaker expects the model artifact to be uploaded in S3, so we upload the tar artiface to S3

In [14]:
s3_code_artifact_accelerate = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

We then proceed to retrieve the LMI container image and create the model using the S3 Code artifacrs and container image URI. 

In [15]:
inference_image_uri = sagemaker.image_uris.retrieve(
    "djl-deepspeed", region=region, version="0.23.0"
)

model_name_acc = name_from_base(f"lora-llama7b")

create_model_response = sm_client.create_model(
    ModelName=model_name_acc,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact_accelerate},
)
model_arn = create_model_response["ModelArn"]

We will be deploying this endpoint to a g5.2xlarge instnace. 

In [17]:
endpoint_config_name = f"{model_name_acc}-config"
endpoint_name = f"{model_name_acc}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name_acc,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
            # "VolumeSizeInGB": 512
        },
    ],
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:917092859813:endpoint-config/lora-llama7b-2023-10-26-15-10-06-286-config".

In [18]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)

In [19]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:917092859813:endpoint/lora-llama7b-2023-10-26-15-10-06-286-endpoint
Status: InService


## Make Inference Requests

We show how you can make requests targetting each of the adapters configured in the endpoint, as well as requests targetting just the base model with no adapters.

Inference Request targetting the tloen/alpaca-lora-7b adapter (named eng_alpaca based on the directory name of the adapter).

In [38]:
%%time

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"inputs": "What is AWS re:invent, where and where does this event happen?", 
                     "adapter_name": "lora2"}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 4.84 ms, sys: 0 ns, total: 4.84 ms
Wall time: 1.51 s


'"AWS Re:Invent is an annual conference hosted by Amazon Web Services in Las Vegas.</s>"'

Inference Request targetting the 22h/cabrita-lora-v0-1 adapter (named portuguese_alpaca based on the direcotry name of the adapter).

In [43]:
%%time

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": "Translate to english: Estou na AWS re:invent e estou gostando",
            "adapter_name": "lora2",
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 4.5 ms, sys: 0 ns, total: 4.5 ms
Wall time: 8.5 s


'"Translated: I\'m at AWS re-invent and I like it.</s>"'

Inference Request targetting the base model without any adapters

In [36]:
%%time

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"inputs": "Tell me about Alpacas"}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 1.88 ms, sys: 3.76 ms, total: 5.64 ms
Wall time: 674 ms


'"Alpacas are a type of animal.</s>"'

## Clean up Resources

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name_acc)